### Import python library

In [1]:
import tensorflow as tf
import numpy as np

from tensorflow import keras
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.models import Sequential
# from tensorflow.keras.utils import np_utils
from tensorflow.keras.layers import Dense, LSTM, Dropout
import tensorflow.keras.backend as K 

from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras.callbacks import EarlyStopping

from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler


import pandas as pd

import matplotlib.pyplot as plt
# import seaborn as sns
# import plotly.express as px
# import plotly.graph_objects as go

from math import sqrt

### 1. Read data


In [2]:
timestep = 3

In [3]:
PCT_lstm_train_X = pd.read_csv("ERA_lstm_final_train_X33.csv")
PCT_lstm_train_y = pd.read_csv("ERA_lstm_final_train_y33.csv")

PCT_lstm_test_X = pd.read_csv("ERA_lstm_final_test_X33.csv")
PCT_lstm_test_y = pd.read_csv("ERA_lstm_final_test_y33.csv")

In [4]:
team = list(PCT_lstm_train_X.T_ID.unique())
year = list(PCT_lstm_train_y.YEAR.unique())

In [5]:
team, year

(['LG', 'HH', 'NC', 'HT', 'SK', 'KT', 'WO', 'LT', 'SS', 'OB'],
 [2016, 2017, 2018, 2019])

#### 4) input shape로 변경 (row, timestep=2, feature)

ex) 
timestep = 2

* X_train_v 구성예시: [[1 ~ 24경기 데이터, 25 ~ 48경기 데이터], [49 ~ 72경기 데이터, 73 ~ 96경기 데이터] ]  
X_train_v.shape >> (2,2*x)             # x: 각 24group에 대한 변수 개수
* y_train_v 구성예시: 97 ~ 120 경기 승률

=> reshape

* X_train_v.shape >> (2,2,x)  # row, timestep, feature

### 2. 모델 구성(LSTM)
- optimizer: Adam -> lr(learning rate) 조절
- LSTM: 모델이 계속 동일한 결과값이 나올 때, input 뉴런 개수를 늘려야 한다는 글을 읽고 계속 input 노드 개수를 바꿔주면서 모델 생성중
- loss: MSE

- early_stop: patience를 크게하면 과적합 되는 경우가 있어서 최대한 작게 설정해둠
- batch_size: 모델이 계속 동일한 결과값이 나올 때, 데이터가 적어 batch size를 줄여보라는 글을 읽고 1로 설정해둠

In [6]:
model_dict = dict()
hist_dict = dict()
test_pred_df = pd.DataFrame([],columns = ['YEAR','T_ID','y','y_pred',"shift_1","shift_2","shift_3",'rms','rms_avg'])

idx = 0
for y in year:
    tmp1 = PCT_lstm_train_X[PCT_lstm_train_X["YEAR"] == y]
    tmp2 = PCT_lstm_train_y[PCT_lstm_train_y["YEAR"] == y]
    tmp3 = PCT_lstm_test_X[PCT_lstm_test_X["YEAR"] == y]
    tmp4 = PCT_lstm_test_y[PCT_lstm_test_y["YEAR"] == y]
    for t in team:
        name = '{}{}'.format(y,t)
        print(name,"=======================================")
        
        X_train = tmp1[tmp1["T_ID"] == t].drop(["T_ID","YEAR"],axis = 1)
        y_train = tmp2[tmp2["T_ID"] == t].drop(["T_ID","YEAR"],axis=1)
        X_test = tmp3[tmp3["T_ID"] == t].drop(["T_ID","YEAR"],axis=1)
        y_test = tmp4[tmp4["T_ID"] == t].drop(["T_ID","YEAR"],axis=1)
        
        X_train_v = X_train.values
        y_train_v = y_train.values

        X_test_v = X_test.values
        y_test_v = y_test.values
        
        X_train_t = X_train_v.reshape(X_train_v.shape[0], timestep,X_train_v.shape[1]//timestep)
        X_test_t = X_test_v.reshape(X_test_v.shape[0], timestep,X_test_v.shape[1]//timestep)
        
        
        ## model
        K.clear_session() 

        model = Sequential()
        optimizer = Adam(lr=0.01)
#         optimizer = RMSprop(lr=0.01, rho=0.9, epsilon=None, decay=0.0)

        model.add(LSTM(100,input_shape = (timestep,X_train_v.shape[1]//timestep))) # (timestep, feature)
        model.add(Dense(1)) # output = 1
        model.compile(loss='mean_squared_error', optimizer=optimizer,metrics=['mae'])

        model.summary()
        
#         hist1 = model.fit(X_train_t, y_train_v, epochs=100, batch_size=1, verbose=1)
        
        early_stop = EarlyStopping(monitor='loss', mode = 'min',patience=2, verbose=1)

        hist1 = model.fit(X_train_t, y_train_v, epochs=100,
                  batch_size=10, verbose=1, callbacks=[early_stop])
        ##
        
        model_dict[name] = model
        hist_dict[name] = hist1
        
        y_pred = model.predict(X_test_t)
        rms = sqrt(mean_squared_error(y_test_v, y_pred))
        rms05 = sqrt(mean_squared_error(y_test_v,[y_train.mean()[0]]))
        
        
        test_pred_df.loc[idx,:] = [y,t,y_test_v.reshape(-1)[0],y_pred.reshape(-1)[0],
                                  X_test.loc[X_test.index[0],["shift_ERA_1"]][0],
                                  X_test.loc[X_test.index[0],["shift_ERA_2"]][0],
                                X_test.loc[X_test.index[0],["shift_ERA_3"]][0],
                                rms,rms05]

        idx += 1

test_pred_df[['y','y_pred',"shift_1","shift_2","shift_3",'rms','rms_avg']] = test_pred_df[['y','y_pred',"shift_1","shift_2","shift_3",'rms','rms_avg']].astype(float)

2016LG =======================================
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 100)               58400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 58,501
Trainable params: 58,501
Non-trainable params: 0
_________________________________________________________________
Train on 25 samples
Epoch 1/100
25/25 [==============================] - 3s 102ms/sample - loss: 11.4810 - mae: 2.7609
Epoch 2/100
25/25 [==============================] - 0s 558us/sample - loss: 1.1127 - mae: 0.9849
Epoch 3/100
25/25 [==============================] - 0s 479us/sample - loss: 0.4248 - mae: 0.5637
Epoch 4/100
25/25 [==============================] - 0s 519us/sample - loss: 0.2057 - mae: 0.3329
Epoch 5/100
25/25 [==============================] - 0

25/25 [==============================] - 2s 86ms/sample - loss: 7.4793 - mae: 2.2034
Epoch 2/100
25/25 [==============================] - 0s 479us/sample - loss: 1.4060 - mae: 1.1167
Epoch 3/100
25/25 [==============================] - 0s 519us/sample - loss: 0.1478 - mae: 0.3139
Epoch 4/100
25/25 [==============================] - 0s 600us/sample - loss: 0.4438 - mae: 0.5724
Epoch 5/100
25/25 [==============================] - 0s 600us/sample - loss: 0.2900 - mae: 0.4446
Epoch 00005: early stopping
2016LT =======================================
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 100)               58400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 58,501
Trainable params: 58,501
Non-trainable params: 0
____________________________

25/25 [==============================] - 0s 519us/sample - loss: 0.2434 - mae: 0.4046
Epoch 6/100
25/25 [==============================] - 0s 556us/sample - loss: 0.2476 - mae: 0.4312
Epoch 7/100
25/25 [==============================] - 0s 597us/sample - loss: 0.2850 - mae: 0.4551
Epoch 00007: early stopping
2017HT =======================================
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 100)               58400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 58,501
Trainable params: 58,501
Non-trainable params: 0
_________________________________________________________________
Train on 25 samples
Epoch 1/100
25/25 [==============================] - 2s 85ms/sample - loss: 15.4397 - mae: 3.2170
Epoch 2/100
25/25 [====================

25/25 [==============================] - 0s 670us/sample - loss: 0.2377 - mae: 0.3753
Epoch 7/100
25/25 [==============================] - 0s 638us/sample - loss: 0.1864 - mae: 0.3406
Epoch 8/100
25/25 [==============================] - 0s 678us/sample - loss: 0.2437 - mae: 0.4184
Epoch 9/100
25/25 [==============================] - 0s 678us/sample - loss: 0.2402 - mae: 0.4264
Epoch 00009: early stopping
2017OB =======================================
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 100)               58400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 58,501
Trainable params: 58,501
Non-trainable params: 0
_________________________________________________________________
Train on 25 samples
Epoch 1/100
25/25 [====================

25/25 [==============================] - 2s 87ms/sample - loss: 13.4680 - mae: 2.9632
Epoch 2/100
25/25 [==============================] - 0s 678us/sample - loss: 1.7706 - mae: 1.2415
Epoch 3/100
25/25 [==============================] - 0s 598us/sample - loss: 0.2222 - mae: 0.4004
Epoch 4/100
25/25 [==============================] - 0s 598us/sample - loss: 0.3286 - mae: 0.4773
Epoch 5/100
25/25 [==============================] - 0s 638us/sample - loss: 0.2894 - mae: 0.4583
Epoch 00005: early stopping
2018WO =======================================
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 100)               58400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 58,501
Trainable params: 58,501
Non-trainable params: 0
___________________________

25/25 [==============================] - 0s 678us/sample - loss: 0.1804 - mae: 0.3637
Epoch 7/100
25/25 [==============================] - 0s 838us/sample - loss: 0.0725 - mae: 0.2149
Epoch 8/100
25/25 [==============================] - 0s 758us/sample - loss: 0.1298 - mae: 0.3081
Epoch 9/100
25/25 [==============================] - 0s 678us/sample - loss: 0.1331 - mae: 0.3121
Epoch 00009: early stopping
2019NC =======================================
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 100)               58400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 58,501
Trainable params: 58,501
Non-trainable params: 0
_________________________________________________________________
Train on 25 samples
Epoch 1/100
25/25 [====================

25/25 [==============================] - 0s 678us/sample - loss: 0.1560 - mae: 0.3419
Epoch 00005: early stopping
2019SS =======================================
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 100)               58400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 58,501
Trainable params: 58,501
Non-trainable params: 0
_________________________________________________________________
Train on 25 samples
Epoch 1/100
25/25 [==============================] - 5s 191ms/sample - loss: 16.4162 - mae: 3.1322
Epoch 2/100
25/25 [==============================] - 0s 758us/sample - loss: 1.3804 - mae: 1.0588
Epoch 3/100
25/25 [==============================] - 0s 718us/sample - loss: 0.5309 - mae: 0.6473
Epoch 4/100
25/25 [===================

In [7]:
mean_squared_error(test_pred_df['y'],test_pred_df['y_pred'])

0.9502917436942981

In [15]:
test_pred_df.groupby(["YEAR"]).mean()

,y,y_pred,shift_1,shift_2,shift_3,rms,rms_avg
YEAR,,,,,,,
2016,4.972786,5.353184,5.462381,5.436733,5.259982,0.616480,0.760568
2017,5.225316,4.889280,4.865418,5.241630,5.487222,0.713289,0.767047
2018,5.383612,5.304931,5.780409,4.984139,5.188945,0.823567,0.731751
2019,3.400941,4.248198,4.366891,4.444551,3.897384,0.996904,1.049148


In [16]:
test_pred_df

,YEAR,T_ID,y,y_pred,shift_1,shift_2,shift_3,rms,rms_avg
0,2016,LG,3.649770,4.607648,5.400000,5.551402,5.584759,0.957878,1.022759
1,2016,HH,5.258114,5.664038,6.412500,5.010736,4.764706,0.405924,0.697619
2,2016,NC,3.575342,4.859971,5.086342,5.057143,5.059908,1.284628,1.383954
3,2016,HT,4.120827,5.159933,5.957746,4.779503,5.551402,1.039106,1.281339
4,2016,SK,5.144882,5.273207,5.646226,5.391576,4.285714,0.128325,0.431617
5,2016,KT,6.314516,6.479980,5.736422,7.130435,6.138249,0.165464,0.599439
6,2016,WO,5.704839,4.713511,4.584112,5.039454,5.803738,0.991328,1.177294
7,2016,LT,5.849294,5.660211,5.314286,5.869565,5.695312,0.189082,0.347195
8,2016,SS,5.400000,5.934407,6.214286,5.070229,5.986047,0.534407,0.127851
9,2016,OB,4.710280,5.178938,4.271889,5.467290,3.729984,0.468658,0.536609


In [20]:
from sklearn.metrics import r2_score
tmp = test_pred_df.copy()
# tmp['half']= 0.5
r2_y_predict = r2_score(tmp['y'], tmp['y_pred'])
r2_y_predict

0.0946381804538674